In [29]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [32]:
phone = Phone.objects.first()
mdata = phone.measurecalldata_set.first()
mdata

<MeasureCallData: 1058076715/5G/20211101091241230/1/None/0.0/>

In [34]:
def rsrp2color(x):
    if x > -65:
        color = 'red'
    elif -75 <= x < -65:
        color = 'orange'
    elif -85 <= x < -75:
        color = 'yellow'
    elif -95 <= x < -85:
        color = 'green'
    elif -105 <= x < -95:
        color = 'blue'
    else:
        color = 'black'
    return color

In [43]:
'''측정위치로 지도를 작성하는 함수'''
# if locations and len(locations) < 1: return None
map = folium.Map(location=[mdata.phone.latitude, mdata.phone.longitude], zoom_start=15)

# 첫번째 측정위치를 지도맵에 표시한다.
folium.Marker(
    location=[mdata.phone.latitude, mdata.phone.longitude],
    icon=folium.Icon(color="red", icon="star"),
    icon_size=(10,10),
).add_to(map)

# 측정 위치를 지도맵에 표시한다.
locations = []
points = folium.FeatureGroup(name="All Points")
qs = mdata.phone.measurecalldata_set.filter(testNetworkType='speed').order_by("meastime")
if qs.exists():
    for m in qs:
        # 네트워크 유형(5G, LTE, 3G)에 따라서 무선품질 정보를 가져온다.
        if m.networkId == '5G':
            pci, RSRP, SINR = m.NR_PCI, m.NR_RSRP, m.NR_SINR
        else:
            pci, RSRP, SINR= m.p_pci, m.p_rsrp, m.p_SINR
        # RSRP 값에 따라서 색상을 계산한다.
        if m.latitude == mdata.latitude and m.longitude == mdata.longitude:
            radius = 40
        else:
            radius = 25
        df = pd.DataFrame([{'PCI': pci, 'Cell ID': m.cellId, 'DL': m.downloadBandwidth, 'UL': m.uploadBandwidth, \
                            'RSRP': RSRP, 'SINR': SINR },])
        html = df.to_html(index=False, 
                        classes='table table-striped table-hover table-condensed table-responsive text-center')
        html = html.replace('<th>', '<td align="center">') # 항목명 가운데 정렬
        html = html.replace('<td>', '<td align="center">') # 항목값 가운데 정렬
        popup = folium.Popup(html, min_width=100, max_width=300)
        folium.Circle(
            location=[m.latitude, m.longitude],
            popup=popup,
            radius=radius, # 크기 지정
            color='black', # 테두리 색상
            fill_color=rsrp2color(RSRP) if RSRP else 'black', # 내부 색상 '#000000' 
            fill_opacity=1.,
            weight=1
        ).add_to(points)
        locations.append([m.latitude, m.longitude])

    # 지도맵에 이동경로를 표시한다.
    folium.PolyLine(locations=locations).add_to(map)

    # 지도맵에 측정지점들을 표시한다.
    # 이동경로와 겹쳐서 먼저 이동경로를 그리고 난 후 측점지점들을 표시한다.
    points.add_to(map)

    # 지도 자동줌 기능(모든 POT과 시설이 지도상에 보여질 수 있도록 자동확대)
    sw = pd.DataFrame(locations).min().values.tolist()
    ne = pd.DataFrame(locations).max().values.tolist()
    map.fit_bounds([sw, ne])

filename = f'{mdata.phone.measdate}-{mdata.ispId}-{mdata.phone_no}.html'
map.save("monitor/templates/maps/" + filename)
map

In [28]:
from ipyleaflet import *
center = [35.16640559, 128.03997589]
zoom = 15
m = Map(default_tiles=TileLayer(opacity=1.0), center=center, zoom=zoom)
map = folium.TileLayer('openstreetmap')
map.add_to(m)
display(map)

AttributeError: 'Map' object has no attribute 'add_child'

In [45]:
import folium
from IPython.display import display
LDN_COORDINATES = (51.5074, 0.1278)
myMap = folium.Map(location=LDN_COORDINATES, zoom_start=12)
display(myMap)

In [46]:
import folium

london_location = [51.507351, -0.127758]

m = folium.Map(location=london_location, zoom_start=15)
m

In [47]:
m.save("test.html)")

In [6]:
import os
from selenium import webdriver

mapFname = 'output.html'
mapUrl = 'file://{0}/{1}'.format(os.getcwd(), mapFname)

# download gecko driver for firefox from here - https://github.com/mozilla/geckodriver/releases

# use selenium to save the html as png image
driver = webdriver.Firefox()
driver.get(mapUrl)
# wait for 5 seconds for the maps and other assets to be loaded in the browser
time.sleep(5)
driver.save_screenshot('output.png')
driver.quit()

SessionNotCreatedException: Message: Expected browser binary location, but unable to find binary in default location, no 'moz:firefoxOptions.binary' capability provided, and no binary flag set on the command line


In [5]:
import folium
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.options import Options
import os
import time

# create a map object with a desired initial map center and initial map zoom
mapObj = folium.Map(location=[24.2170111233401, 81.0791015625000],
                    zoom_start=5)

# draw some circles
circlesData = [
    [25, 74, 80000],
    [22, 79, 60000],
    [26, 82, 90000]
]

for d in circlesData:
    folium.Circle(radius=d[2],
                  location=[d[0], d[1]],
                  fill=True,
                  stroke=False,
                  fill_opacity=0.6,
                  ).add_to(mapObj)

# save the map as html
mapFname = 'output.html'
mapObj.save(mapFname)

mapUrl = 'file://{0}/{1}'.format(os.getcwd(), mapFname)
print(mapUrl)

# download gecko driver for firefox from here - https://github.com/mozilla/geckodriver/releases

# use selenium to save the html as png image
# driver = webdriver.Firefox("/Users/jonghyunkim/code/smartproject/")
# binary = FirefoxBinary("/Users/jonghyunkim/code/smartproject/geckodriver")
# driver = webdriver.Firefox(firefox_binary=binary)
options = Options()
options.binary_location = r"/opt/homebrew/bin/geckodriver"
driver = webdriver.Firefox(executable_path=r"/opt/homebrew/bin/geckodriver", options=options)
# driver.get(mapUrl)
driver.get("http://localhost:8888/view/output.html")
# wait for 5 seconds for the maps and other assets to be loaded in the browser
time.sleep(5)
driver.save_screenshot('output.png')
driver.quit()

file:///Users/jonghyunkim/code/smartproject/output.html


/var/folders/dw/slp8d8cn6757kmxq7587c3n00000gn/T/ipykernel_43429/1186900133.py:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r"/opt/homebrew/bin/geckodriver", options=options)


InvalidArgumentException: Message: binary is not a Firefox executable


In [43]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
import time
from selenium.webdriver.firefox.options import Options

# Create a new instance of the Firefox driver
# options = Options()
# options.binary_location = r"/opt/homebrew/bin/geckodriver"
# driver = webdriver.Firefox(executable_path=r"/opt/homebrew/bin/geckodriver", options=options)
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
caps = DesiredCapabilities.FIREFOX
caps['marionette'] = True
caps['binary'] = r"/opt/homebrew/bin/geckodriver"
d = webdriver.Firefox(capabilities=caps)


# go to the google home page
driver.get("http://www.google.com")

# the page is ajaxy so the title is originally this:
print(driver.title)

# find the element that's name attribute is q (the google search box)
inputElement = driver.find_element_by_name("q")

# type in the search
inputElement.send_keys("cheese!")

# submit the form (although google automatically searches now without submitting)
inputElement.submit()
time.sleep(5)

try:
    # we have to wait for the page to refresh, the last thing that seems to be updated is the title
    WebDriverWait(driver, 10).until(EC.title_contains("cheese!"))

    # You should see "cheese! - Google Search"
    print(driver.title)
    time.sleep(5)

finally:
    driver.quit()

/var/folders/dw/slp8d8cn6757kmxq7587c3n00000gn/T/ipykernel_50392/364796821.py:16: DeprecationWarning: capabilities and desired_capabilities have been deprecated, please pass in a Service object
  d = webdriver.Firefox(capabilities=caps)


InvalidArgumentException: Message: binary is not a Firefox executable


In [ ]:
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('image.png')

https://github.com/mozilla/geckodriver/releases
https://www.youtube.com/watch?v=RL-phs5FnmU
https://dejavuqa.tistory.com/106

In [54]:
import sys
print (sys.argv[-1])

django_extensions.management.notebook_extension


In [59]:
from django.contrib.sites.models import Site

current_site = Site.objects.get_current()


RuntimeError: Model class django.contrib.sites.models.Site doesn't declare an explicit app_label and isn't in an application in INSTALLED_APPS.

In [3]:
import pandas as pd

pd.DataFrame([[1,2],[3,4]]).min().values.tolist()

[1, 2]

In [7]:
mapObj = folium.Map(location=[24.2170111233401, 81.0791015625000],
                    zoom_start=5)
mapObj

In [26]:
[ int(x) for x in ReportCycle.objects.all()[0].reportCycle.split(',')]

[3, 10, 27, 37, 57]